In [50]:
import pandas as pd

In [51]:
raw_energy_df = pd.read_csv('./data/raw/energy_dataset.csv')
raw_weather_df = pd.read_csv('./data/raw/weather_features.csv')

In [52]:
raw_weather_df = raw_weather_df.rename(columns={'clouds_all': 'cloud_cover_pct'})
raw_weather_df['precipitation_last_3hr'] = raw_weather_df['snow_3h'] + raw_weather_df['rain_3h']
raw_weather_df = raw_weather_df.drop(columns=['temp_min', 'temp_max', 'wind_deg', 'snow_3h', 'rain_1h', 'rain_3h', 'weather_id', 'weather_main', 'weather_description', 'weather_icon', 'pressure', 'humidity'])

valencia_df = raw_weather_df[raw_weather_df['city_name'] == 'Valencia']
madrid_df = raw_weather_df[raw_weather_df['city_name'] == 'Madrid']
bilbao_df = raw_weather_df[raw_weather_df['city_name'] == 'Bilbao']
barcelona_df = raw_weather_df[raw_weather_df['city_name'] == ' Barcelona']
seville_df = raw_weather_df[raw_weather_df['city_name'] == 'Seville']
dfs = {
    'valencia': valencia_df,
    'madrid': madrid_df,
    'bilbao': bilbao_df,
    'barcelona': barcelona_df,
    'seville': seville_df
}

renamed_dfs = []
for city, df in dfs.items():
    df = df.rename(columns={col: f"{city}_{col}" for col in df.columns if col != 'dt_iso'})
    dfs[city] = df

out1 = pd.merge(dfs['valencia'], dfs['madrid'], on='dt_iso', how='inner')
out2 = pd.merge(out1, dfs['bilbao'], on='dt_iso', how='inner')
out3 = pd.merge(out2, dfs['barcelona'], on='dt_iso', how='inner')
out4 = pd.merge(out3, dfs['seville'], on='dt_iso', how='inner')
weather_df = out4
weather_df = weather_df.drop(columns=['valencia_city_name', 'madrid_city_name', 'barcelona_city_name', 'bilbao_city_name', 'seville_city_name'])
del out1, out2, out3, out4, dfs

In [53]:
energy_df = raw_energy_df.drop(columns=['generation fossil coal-derived gas', 'generation fossil oil shale', 'generation fossil peat', 'generation geothermal', 'generation wind offshore', 'generation marine', 'forecast wind offshore eday ahead', 'generation hydro pumped storage aggregated', 'price day ahead', 'total load forecast'])
energy_df = energy_df.rename(columns={'total load actual': 'total load'})
energy_df = energy_df.dropna().copy()
energy_df['dt_iso'] = energy_df['time']
energy_df['generation wind'] = energy_df['generation wind onshore']
energy_df['generation water'] = energy_df[['generation hydro pumped storage consumption','generation hydro run-of-river and poundage', 'generation hydro water reservoir']].sum(axis=1)
energy_df['generation solar'] = energy_df['generation solar']
energy_df['generation other'] = energy_df[['generation biomass', 'generation nuclear', 'generation other renewable', 'generation fossil brown coal/lignite', 'generation fossil gas', 'generation fossil hard coal','generation fossil oil', 'generation waste', 'generation other']].sum(axis=1)
energy_df['generation total'] = energy_df[['generation wind', 'generation water', 'generation solar', 'generation other']].sum(axis=1)
energy_df = energy_df[['dt_iso', 'generation wind', 'generation water', 'generation solar', 'generation total', 'total load', 'price actual']]

In [54]:
weather_df.to_csv('./data/clean/weather.csv', index=False)
energy_df.to_csv('./data/clean/energy.csv', index=False)